In [1]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

IMDB Tokenization

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x1d22b0c1580>)

In [3]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [4]:
print("Number of Reviews : ",len(train_df))

Number of Reviews :  50000


In [5]:
#센텐스피스의 입력으로 사용하기 위해서 데이터프레임을 txt 파일로 저장
with open('imdb_review.txt','w',encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [6]:
#단어 집합과 각 단어에 고유한 정수 부여(정수 인코딩)
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [7]:
#vocab 파일에서는 학습된 서브 워드들을 확인 할 수 있음
vocab_list = pd.read_csv('imdb.vocab',sep='\t',header=None,quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
3572,ivil,-3569
3633,▁magn,-3630
367,lf,-364
4255,▁thus,-4252
1753,na,-1750
2106,▁including,-2103
3902,▁bi,-3899
1249,▁Mar,-1246
4038,▁Wil,-4035
1930,ctor,-1927


In [8]:
#단어 집합 크기
print(len(vocab_list))

5000


In [9]:
#model 파일을 이용한 디코딩 작업
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

In [10]:
#encode_as_pieces : 문장을 입력하면 서브 워드 시퀀스로 변환합니다.
#encode_as_ids : 문장을 입력하면 정수 시퀀스로 변환합니다.

lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



In [11]:
#GetPieceSize() : 단어 집합의 크기를 확인합니다.
sp.GetPieceSize()

5000

In [12]:
#idToPiece : 정수로부터 맵핑되는 서브 워드로 변환합니다.
sp.IdToPiece(500)

'▁ent'

In [15]:
#PieceToId : 서브워드로부터 맵핑되는 정수로 변환합니다.
sp.PieceToId('▁ent')

500

In [16]:
#DecodeIds : 정수 시퀀스로부터 문장으로 변환합니다.
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

'I have waited a long time for someone to film'

In [17]:
#DecodePieces : 서브워드 시퀀스로부터 문장으로 변환합니다.
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

In [18]:
#encode : 문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환 가능합니다.
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]


네이버 영화 리뷰 토큰화 

In [19]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x1d24d3cf9a0>)

In [20]:
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [21]:
print('리뷰 개수: ',len(naver_df))

리뷰 개수:  200000


In [22]:
#null 값 제거
print(naver_df.isnull().values.any())

True


In [23]:
naver_df = naver_df.dropna(how='any') #null 값이 존재하는 행 제거
print(naver_df.isnull().values.any())

False


In [24]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

리뷰 개수 : 199992


In [25]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [26]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [27]:
#naver.vocab 불러오기
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)

In [28]:
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,▁그,-6


In [29]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [31]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]

for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[132, 966, 1296, 2590, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]



In [33]:
sp.IdToPiece(4)

'영화'

In [34]:
sp.PieceToId('영화')

4

In [35]:
sp.DecodeIds([54, 200, 821, 85])

'진짜 최고의 영화입니다 ᄏᄏ'

In [36]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])

'진짜 최고의 영화입니다 ᄏᄏ'

In [37]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]
